In [15]:
from rich.console import Console
import sys
import builtins

console = Console(force_terminal=False, file=sys.__stdout__)

builtins.print = console.print

In [16]:
from pydantic import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

In [17]:
class Sentiment(BaseModel):
    """Define o sentimento e o idioma da mensagem enviada"""
    sentimento: str = Field(description="Sentimento do Texto. Deve ser 'pos', 'neg' ou 'nt' para não definido")
    lingua: str = Field(description="Língua que o texto foi escrito (deve estar no charset UTF-8)")
    
tool_sentimento = convert_to_openai_function(Sentiment)
print(tool_sentimento)

{
    'name': 'Sentiment',
    'description': 'Define o sentimento e o idioma da mensagem enviada',
    'parameters': {
        'properties': {
            'sentimento': {
                'description': "Sentimento do Texto. Deve ser 'pos', 'neg' ou 'nt' para não definido",
                'type': 'string'
            },
            'lingua': {
                'description': 'Língua que o texto foi escrito (deve estar no charset UTF-8)',
                'type': 'string'
            }
        },
        'required': ['sentimento', 'lingua'],
        'type': 'object'
    }
}

In [18]:
texto = "Eu gosto muito de pizza nordestina"

In [19]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "Analise o texto e categorize-o conforme as instruções"),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = prompt | chat.bind(functions=[tool_sentimento], 
                           function_call={"name": "Sentiment"})

In [20]:
print(chain.invoke({"input": texto}))

AIMessage(
    content='',
    additional_kwargs={
        'function_call': {'arguments': '{"sentimento":"pos","lingua":"pt"}', 'name': 'Sentiment'},
        'refusal': None
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 11,
            'prompt_tokens': 137,
            'total_tokens': 148,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo-0125',
        'system_fingerprint': None,
        'id': 'chatcmpl-BrMO95b20dzto8f1OPABLKnxVE2so',
        'service_tier': 'default',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--ef8bc4ba-ef72-45ea-b3f8-17c7aa9d4844-0',
    usage_metadata={
        'input_tokens': 137,
        'output_tokens': 11,
        'total_tokens': 148,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 0}
    }
)

In [21]:
print(chain.invoke({"input": "May the Force be with you"}))

AIMessage(
    content='',
    additional_kwargs={
        'function_call': {'arguments': '{"sentimento":"pos","lingua":"en"}', 'name': 'Sentiment'},
        'refusal': None
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 11,
            'prompt_tokens': 134,
            'total_tokens': 145,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo-0125',
        'system_fingerprint': None,
        'id': 'chatcmpl-BrMOaxwsoY4Ao0rz8bquE7v70DOM4',
        'service_tier': 'default',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--edc1c7c1-e49e-4303-8484-92f5ebb1ab93-0',
    usage_metadata={
        'input_tokens': 134,
        'output_tokens': 11,
        'total_tokens': 145,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 0}
    }
)

In [10]:
solicitacoes = [
    "Meu computador está travando toda vez que tento abrir um programa. O que devo fazer?",
    "Não consigo acessar minha conta. A senha não está funcionando!",
    "Estou tendo dificuldades para instalar o software de design gráfico. Pode me ajudar?",
    "Gostaria de saber como atualizar meu endereço de entrega na plataforma.",
    "O meu laptop não está ligando. Acho que é um problema no cabo de energia.",
    "Não consigo ver meus arquivos no aplicativo. Existe alguma solução?"
]

In [11]:
from enum import Enum

class SetorEnum(str, Enum):
    suporte_hardware = "suporte_hardware"
    suporte_software = "suporte_software"
    suporte_conta = "suporte_conta"
    outros = "outros"
    
class DirecionaSuporte(BaseModel):
    """Direciona a solicitação de suporte para o setor responsável"""
    setor: SetorEnum

In [22]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_direcionamento_suporte = convert_to_openai_function(DirecionaSuporte)
print(tool_direcionamento_suporte)

{
    'name': 'DirecionaSuporte',
    'description': 'Direciona a solicitação de suporte para o setor responsável',
    'parameters': {
        'properties': {
            'setor': {
                'enum': ['suporte_hardware', 'suporte_software', 'suporte_conta', 'outros'],
                'type': 'string'
            }
        },
        'required': ['setor'],
        'type': 'object'
    }
}

In [24]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Criar o prompt com as instruções detalhadas para o modelo
system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
Questões relacionadas a problemas de hardware, como falha no computador, laptop ou 
outros dispositivos físicos devem ser direcionadas para "suporte_hardware".
Questões relacionadas a problemas com software, como instalação, erros ao abrir programas, 
etc., devem ser direcionadas para "suporte_software".
Problemas relacionados ao acesso de conta, como recuperação de senha, problemas de login, 
devem ser direcionados para "suporte_conta".
Mensagens que não se encaixam nessas categorias devem ser direcionadas para "outros".
'''

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_direcionamento_suporte],
                            function_call={"name":"DirecionaSuporte"})
         | JsonOutputFunctionsParser())

In [25]:
solicitacao = solicitacoes[0]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

Solicitação: Meu computador está travando toda vez que tento abrir um programa. O que devo fazer?

Resposta: {'setor': 'suporte_software'}

In [26]:
solicitacao = solicitacoes[3]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

Solicitação: Gostaria de saber como atualizar meu endereço de entrega na plataforma.

Resposta: {'setor': 'suporte_conta'}

In [27]:
solicitacao = solicitacoes[4]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

Solicitação: O meu laptop não está ligando. Acho que é um problema no cabo de energia.

Resposta: {'setor': 'suporte_hardware'}